In [1]:
import os
%cd ..
print(f"Current working directory: {os.getcwd()}")

d:\Projects\tiny_llm
Current working directory: d:\Projects\tiny_llm


# TinyLLM: A Hands-On Introduction to AI/ML and LLMs

This Jupyter Notebook serves as a practical guide for students and enthusiasts to delve into the fundamentals of Artificial Intelligence (AI), Machine Learning (ML), Neural Networks (NN), and Large Language Models (LLMs) by building a smaller, more manageable model.

Our focus is on creating a "TinyLLM" using the **TinyStories dataset**. This dataset is specifically designed to be small and simple, making it ideal for:
*   **Learning Core Concepts:** Understand the end-to-end process of an LLM workflow, from data preparation to training and inference.
*   **Hardware Accessibility:** Train a functional LLM even on consumer-grade hardware, overcoming common barriers for students.
*   **Rapid Experimentation:** Quickly iterate and observe the effects of changes due to faster training times.

By working through this notebook, you will gain hands-on experience with:
*   Setting up a Python environment for deep learning.
*   Loading and processing text data for LLM training.
*   Understanding the architecture of a transformer-based language model.
*   Training an LLM from scratch.
*   Performing interactive text generation (inference).
*   Evaluating the performance of your trained model.

This project is structured to provide a clear, step-by-step learning path, allowing you to grasp complex concepts through practical application.

## 1. Install Dependencies

Before running the model, ensure all necessary Python packages are installed. This project uses `torch`, `transformers`, and `datasets`.

Run the following cell to install dependencies from `requirements.txt`. If you are using a virtual environment, make sure it's activated before launching Jupyter, or specify the full path to your Python executable within the virtual environment.

In [2]:
%pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


## 1.5. Debug Data Loading (Optional)

This cell is for debugging the data loading process. It will help determine if the Hugging Face dataset is being loaded correctly or if the fallback text is being used.

In [2]:
# Ensure we are in the correct directory
%pwd

from src.llm_data import load_and_process_dataset

print("\nAttempting to load and process the dataset...")
try:
    train_data, val_data = load_and_process_dataset()
    print(f"\n--- Data Loading Summary ---")
    print(f"Training data samples: {len(train_data)}")
    print(f"Validation data samples: {len(val_data)}")
    print("---------------------------")
except Exception as e:
    print(f"\nAn error occurred during data loading: {e}")

d:\Projects\tiny_llm\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using CUDA (NVIDIA GPU) for training.

Attempting to load and process the dataset...

Loading pre-trained tokenizer: gpt2...
Set tokenizer pad_token to eos_token.
Tokenizer loaded. Vocabulary Size: 50257
Loading and processing dataset from Hugging Face...
Dataset tokenized.
Dataset processed and grouped.
Training data samples: 1657972
Validation data samples: 184220

--- Data Loading Summary ---
Training data samples: 1657972
Validation data samples: 184220
---------------------------


## 2. Train the TinyLLM Model

This step trains the TinyLLM model using the `src/main_train.py` script. The model will be trained for `100000` iterations on the `TinyStories` dataset and saved to `models/tinystories_llm_v1.pth`.

**Note**: This training process can can take a significant amount of time depending on your hardware (CPU/GPU/MPS).

In [3]:
import subprocess
import sys

# Ensure we are in the correct directory
%pwd

print("Starting TinyLLM model training with real-time output...")

# Construct the command to run the training script
# Use sys.executable to ensure the correct Python interpreter (e.g., from a venv) is used
command = [sys.executable, "-m", "src.main_train"]

# Execute the command using subprocess.Popen for real-time output
# bufsize=1 ensures line-buffered output
process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True, bufsize=1)

# Read and print output line by line in real-time
for line in process.stdout:
    print(line, end='') # end='' prevents extra newlines as print already includes them

# Wait for the process to complete and get the return code
process.wait()

if process.returncode == 0:
    print("\nTraining script finished successfully.")
else:
    print(f"\nTraining script exited with error code {process.returncode}.")

Starting TinyLLM model training with real-time output...
Using CUDA (NVIDIA GPU) for training.

Loading pre-trained tokenizer: gpt2...
Set tokenizer pad_token to eos_token.
Tokenizer loaded. Vocabulary Size: 50257
Loading and processing dataset from Hugging Face...
Dataset tokenized.
Dataset processed and grouped.
Training data samples: 1657972
Validation data samples: 184220
Model initialized with 32,165,969 parameters.
Current working directory: d:\Projects\tiny_llm
Expected model save path: d:\Projects\tiny_llm\models\tinystories_llm_v1.pth

Starting training on cuda...
Step 0: train loss 10.8389, val loss 10.8386, lr 0.000000
  Time Spent: 15.69s
  Resources: CPU: 21.7% | RAM: 35.7% | GPU: 88% | VRAM: 6.53/15.92 GB
Step 500: train loss 0.5258, val loss 0.5251, lr 0.000100
  Time Spent: 0h 1m 18s
  Estimated Total: 0h 21m 11s
  Estimated Remaining: 0h 19m 53s
  Resources: CPU: 23.7% | RAM: 37.0% | GPU: 91% | VRAM: 11.24/15.92 GB
Step 1000: train loss 0.1692, val loss 0.1702, lr 0.00

## 3. Run Interactive Inference

After the model has been successfully trained and saved, you can use the `src/main_inference.py` script to interactively generate text.

The inference script will load the `tinystories_llm_v1.pth` model and prompt you to enter text. The model will then attempt to complete your input. The generation uses a `temperature` of `0.8` for more varied output.

**To exit the interactive session, type `exit` when prompted.**

In [ ]:
!python -m src.main_inference

## 4. Evaluate the TinyLLM Model

This section allows you to evaluate the trained TinyLLM model using the `src/main_eval.py` script. It includes options to test the Gemini API connection, calculate perplexity, generate sample stories, and perform automated LLM-as-a-Judge evaluation.

In [3]:
print("\n--- Testing Gemini API Connection ---")
!python -m src.main_eval --test-connection

print("\n--- Running Full Evaluation ---")
!python -m src.main_eval --perplexity --samples --judge



--- Testing Gemini API Connection ---
Using CUDA (NVIDIA GPU) for training.

--- Testing Gemini API Connection ---
Successfully connected to Gemini API.

--- Running Full Evaluation ---
Using CUDA (NVIDIA GPU) for training.
Loading model and tokenizer...

Loading pre-trained tokenizer: gpt2...
Set tokenizer pad_token to eos_token.
Tokenizer loaded. Vocabulary Size: 50257
Model loaded successfully.
Loading and processing dataset from Hugging Face...
Dataset tokenized.
Dataset processed and grouped.
Training data samples: 1657972
Validation data samples: 184220

Calculating perplexity...
Perplexity on validation set: 1.01

--- Generating Sample Stories ---

Prompt: Once upon a time, in a land far away,
Generated Text:
Once upon a time, in a land far away,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,

Prompt: The little robot woke up and saw
Generated Text:
The little robot woke up and saw saw saw saw saw saw saw saw saw saw saw saw 

---